<a href="https://colab.research.google.com/github/Syamabbas/Logistics-Customer-Insights-Tool/blob/main/Auto_Labeling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [51]:
import pandas as pd
import re
from google.colab import files
from datetime import timedelta
from collections import defaultdict

df = pd.read_excel("/content/drive/MyDrive/Google Collab/Analysis Sentiment/Data Clean/Data clean Jan 2026.xlsx")

In [52]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


import pandas as pd
import re
from datetime import timedelta

REFERENCE_DATE = pd.Timestamp("2025-12-10")

def convert_relative_time(text, ref_date=REFERENCE_DATE):
    if pd.isna(text):
        return None

    text = str(text).lower().strip()

    # hapus kata "diedit"
    text = re.sub(r"\bdiedit\b", "", text).strip()

    # normalisasi kata khusus jadi angka
    special_cases = {
        "sebulan": "1 bulan",
        "setahun": "1 tahun",
        "seminggu": "1 minggu",
        "sehari": "1 hari",
        "sejam": "1 jam",
        "semenit": "1 menit"
    }

    for key, value in special_cases.items():
        if key in text:
            text = text.replace(key, value)

    # regex utama: "2 bulan lalu", "3 hari lalu", dll
    match = re.search(r"(\d+)\s+(menit|jam|hari|minggu|bulan|tahun)\s+lalu", text)
    if not match:
        return None

    value = int(match.group(1))
    unit = match.group(2)

    if unit == "menit":
        return ref_date - timedelta(minutes=value)
    elif unit == "jam":
        return ref_date - timedelta(hours=value)
    elif unit == "hari":
        return ref_date - timedelta(days=value)
    elif unit == "minggu":
        return ref_date - timedelta(weeks=value)
    elif unit == "bulan":
        return ref_date - pd.DateOffset(months=value)
    elif unit == "tahun":
        return ref_date - pd.DateOffset(years=value)

    return None

In [53]:
REFERENCE_DATE = pd.Timestamp("2026-01-10")

def convert_relative_time(text, ref_date=REFERENCE_DATE):
    if pd.isna(text):
        return None

    text = str(text).lower().strip()

    # hapus kata "edited"
    text = re.sub(r"\bedited\b", "", text).strip()

    # normalisasi "a / an" → 1
    text = re.sub(r"\b(a|an)\b", "1", text)

    # mapping unit (singular & plural)
    unit_map = {
        "minute": "menit", "minutes": "menit",
        "hour": "jam", "hours": "jam",
        "day": "hari", "days": "hari",
        "week": "minggu", "weeks": "minggu",
        "month": "bulan", "months": "bulan",
        "year": "tahun", "years": "tahun"
    }

    # regex utama: "2 weeks ago", "3 hours ago", "1 day ago"
    match = re.search(r"(\d+)\s+(minute|minutes|hour|hours|day|days|week|weeks|month|months|year|years)\s+ago", text)
    if not match:
        return None

    value = int(match.group(1))
    unit_en = match.group(2)
    unit = unit_map[unit_en]

    if unit == "menit":
        return ref_date - timedelta(minutes=value)
    elif unit == "jam":
        return ref_date - timedelta(hours=value)
    elif unit == "hari":
        return ref_date - timedelta(days=value)
    elif unit == "minggu":
        return ref_date - timedelta(weeks=value)
    elif unit == "bulan":
        return ref_date - pd.DateOffset(months=value)
    elif unit == "tahun":
        return ref_date - pd.DateOffset(years=value)

    return None

In [54]:
df['tanggal_review'] = df['Waktu'].apply(convert_relative_time)
df['tanggal_review'] = pd.to_datetime(df['tanggal_review']).dt.date

In [55]:
# LOAD KAMUS TYPO
def load_gsheet(sheet_name):
    url = f"https://docs.google.com/spreadsheets/d/12ypDLePiU0f-50wsfsQ8N21aWLjmeb8q5k9L8tn4YIU/gviz/tq?tqx=out:csv&sheet={sheet_name}"
    return pd.read_csv(url)

typo_df = load_gsheet("Kamus_Typo")

# pastikan lowercase & tidak ada NaN
typo_df['Typo'] = typo_df['Typo'].astype(str).str.lower().str.strip()
typo_df['Correction'] = typo_df['Correction'].astype(str).str.lower().str.strip()

# convert ke dictionary
typo_dict = dict(zip(typo_df['Typo'], typo_df['Correction']))

print(f"Total typo loaded: {len(typo_dict)}")

Total typo loaded: 155


In [56]:
# PATTERN NORMALIZATION
patterns = {
    r'(\d+)\s*hri': r'\1 hari',
    r'(\d+)\s*hr': r'\1 hari',
    r'(\d+)\s*d': r'\1 hari'
}

In [57]:
# CLEANING FUNCTIONS
def clean_text(text):
    text = str(text).lower()
    text = re.sub(r'http\S+', '', text)
    text = re.sub(r'[^a-z0-9\s]', ' ', text)
    text = re.sub(r'(.)\1{2,}', r'\1', text)  # huruf berulang
    text = re.sub(r'\s+', ' ', text).strip()
    return text

def normalize_pattern(text):
    for p, rpl in patterns.items():
        text = re.sub(p, rpl, text)
    return text

def normalize_typo(text):
    for typo, correct in typo_dict.items():
        text = re.sub(rf'\b{re.escape(typo)}\b', correct, text)
    return text

In [58]:
# APPLY TO DATAFRAME
df['clean_text'] = df['Komentar'].apply(clean_text)
df['clean_text'] = df['clean_text'].apply(normalize_pattern)
df['clean_text'] = df['clean_text'].apply(normalize_typo)

In [59]:
# LOAD KAMUS SENTIMENT FROM GOOGLE SHEET
sentiment_df = load_gsheet("Kamus_Sentiment")

# normalisasi
sentiment_df['keyword'] = sentiment_df['keyword'].astype(str).str.lower().str.strip()
sentiment_df['sentiment'] = sentiment_df['sentiment'].astype(str).str.lower().str.strip()

# pisahkan positive & negative
positive_words = sentiment_df[sentiment_df['sentiment'] == 'positive']['keyword'].tolist()
negative_words = sentiment_df[sentiment_df['sentiment'] == 'negative']['keyword'].tolist()

print("Positive words:", len(positive_words))
print("Negative words:", len(negative_words))

Positive words: 109
Negative words: 195


In [60]:
def auto_label_with_details(text, pos_words, neg_words):
    text = text.lower()

    found_pos = [
        w for w in pos_words
        if re.search(rf'\b{re.escape(w)}\b', text)
    ]
    found_neg = [
        w for w in neg_words
        if re.search(rf'\b{re.escape(w)}\b', text)
    ]

    pos_count = len(found_pos)
    neg_count = len(found_neg)
    score = abs(pos_count - neg_count)

    if pos_count > neg_count:
        sentiment = "positif"
    elif neg_count > pos_count:
        sentiment = "negatif"
    else:
        sentiment = "netral"

    return sentiment, score, ", ".join(found_pos), ", ".join(found_neg)

In [61]:
# Menerapkan fungsi dan memecah hasilnya ke 4 kolom baru
df[['label', 'confidence', 'positive_word', 'negative_word']] = df['clean_text'].apply(
    lambda x: pd.Series(auto_label_with_details(x, positive_words, negative_words))
)

# Tampilkan beberapa baris pertama untuk pengecekan
df[['Komentar', 'label', 'positive_word', 'negative_word']].head()

,Komentar,label,positive_word,negative_word
0,"Cabang sentral cargo PALING LEMOT, dari jkt ke...",negatif,profesional,"lambat, lemot, tertahan, tidak profesional"
1,Bagus,positif,"bagus, bagus",
2,BINTANG 0 BESAR. Ekspedisi paling lambat yang ...,negatif,,"lambat, tidak direkomendasikan"
3,Jangan kerja kalo ga niat!!!! Uang saya dilari...,negatif,,tidak niat
4,"Tidak amanah sih, di buat langganan tapi malah...",positif,"amanah, langganan, sesuai","tidak amanah, gila"


In [62]:
# Simpan ke Excel
file_name = "/content/drive/MyDrive/Google Collab/Analysis Sentiment/Auto Labeling/auto labeling Jan 2026.xlsx"
df.to_excel(file_name, index=False)

# Download
files.download(file_name)
print("File berhasil dibuat dan sedang diunduh!")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

File berhasil dibuat dan sedang diunduh!
